In [ ]:
import rasterio
import rasterio.plot
import pyproj
import matplotlib
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import osr
import math
from rasterio.plot import show
import fiona
import rasterio
import rasterio.mask
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
import os
import re
import statistics
import pandas as pd
from scipy import stats

In [ ]:
folder=r'D:\Desktop\akshat\Internship_Stuff\Dataset\SPI\Pearson3\spi_6'
print(os.listdir(folder))

In [ ]:
#Changing directory to load files from and save output in
os.chdir(r'D:\Desktop\akshat\Internship_Stuff\Dataset\SPI\Pearson3\spi_6\\')
#Checking if gdal library is loaded properly- sometimes it doesnt load properly
!gdalinfo -nomd spi_06_PearsonIII_X1981_01_water_masked.tif

In [ ]:
rasternames=[]
for file_name in os.listdir(folder):
    rasternames.append(file_name)

In [ ]:
print(rasternames)

In [ ]:
img1=gdal.Open(r'Location for Base raster for attribute extraction')
col=img1.RasterXSize
rows=img1.RasterYSize
driver=img1.GetDriver()
img1_val=img1.GetRasterBand(1).ReadAsArray().flatten()

In [ ]:
mask_in=[]
for i in range(0,len(img1_val)):
    if img1_val[i]!=0:
        mask_in.append(i)
print(len(mask_in))
print(mask_in)

In [ ]:
def masking_1(arr):
    masked_list=[]
    for i in mask_in:
        masked_list.append(arr[i])
    return masked_list

# Extra stuff to make csv easily later

In [ ]:
total=[]
for i in range(0,480):
    total.append(len(mask_in))
print(len(total))

In [ ]:
index=[]
count_year=1981
count_month=1
for i in range(0,480):
    if count_month==13:
        count_month=1
        count_year+=1
    if count_month<10:
        month="0"+str(count_month)
    else:
        month=str(count_month)
    date_val=str(count_year)+"_"+month
    index.append(date_val)
    count_month+=1

In [ ]:
# Function to get Area Stats
def area_stats(abc):
    reslt=[]
    drought_no_drought=0
    mild_drought=0
    moderate_drought=0
    severe_drought=0
    extreme_drought=0
    for i in range(0,len(abc)):
        if abc[i]<0:
            drought_no_drought+=1
        if abc[i]<0 and abc[i]>=-1:
            mild_drought+=1
        if abc[i]<-1 and abc[i]>=-1.5:
            moderate_drought+=1
        if abc[i]<-1.5 and abc[i]>=-2:
            severe_drought+=1
        if abc[i]<-2:
            extreme_drought+=1
    reslt.append(drought_no_drought/6446)
    reslt.append(mild_drought/6446)
    reslt.append(moderate_drought/6446)
    reslt.append(severe_drought/6446)
    reslt.append(extreme_drought/6446)
    return reslt

In [ ]:
values=[]
for i in rasternames:
    img=gdal.Open(i)
    val=img.GetRasterBand(1).ReadAsArray().flatten()
    val=masking_1(val)
    result=area_stats(val)
    values.append(result)

In [ ]:
values_2=[]
for i in range(0,5):
    tem=[]
    for j in values:
        tem.append(j[i])
    values_2.append(tem)
print(values_2)
for i in values_2:
    print(len(i))

In [ ]:
data={
    "total":total,
    "No of pixels that show drought -> spi<0":values_2[0],
    "No of pixels that show mild drought -> -1<spi<0":values_2[1],
    "No of pixels that show moderate drought -> -1.5<spi<-1":values_2[2],
    "No of pixels that show severe drought -> -2<spi<-1.5":values_2[3],
    "No of pixels that show extreme drought -> spi<-2":values_2[4],
}

In [ ]:
df = pd.DataFrame(data, index)

In [ ]:
print(df.head)

In [ ]:
#Save as CSV
df.to_csv("D:\\Desktop\\akshat\\Internship_Stuff\\Dataset\\results\\area_stats_spi_9_pearson3.csv")

In [ ]:
plt.figure(figsize=(24,7))
plt.plot(index[0::6],values_2[4][0::6])
plt.plot(index[0::6],values_2[3][0::6])
plt.plot(index[0::6],values_2[2][0::6])
plt.plot(index[0::6],values_2[1][0::6])
plt.legend(['Extreme Drought','Severe Drought','Moderate Drought','Mild Drought'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=90)
plt.show()

# Start of Shapiro Test

In [ ]:
s_test_val=[]
for i in range(8,len(rasternames)):#Change the starting index based on timescale- SPI1=0, SPI3=2, SPI6=5, SPI9=8 
    img=gdal.Open(rasternames[i])
    val=img.GetRasterBand(1).ReadAsArray().flatten()
    val=masking_1(val)
    s_test_val.append(val)
print(len(s_test_val))

In [ ]:
s_test_val_2=[]
for i in range(0,len(mask_in)):
    pixel_val=[]
    for j in s_test_val:
        pixel_val.append(j[i])
    s_test_val_2.append(pixel_val)
print(len(s_test_val_2))

In [ ]:
shapiro_values=[]
for i in s_test_val_2:
    shapiro_test=stats.shapiro(i)
    shapiro_values.append(shapiro_test.pvalue)

In [ ]:
#Count of pixels with significant value
count=0
for i in shapiro_values:
    if i>0.01:#Change the value based on chosen alpha value
        count+=1
print(count)

In [ ]:
def area_show(abc):
    retrn=[]
    for i in range(0,len(abc)):
        if abc[i]>0.01:
            retrn.append(1)
        else:
            retrn.append(0)
    return retrn

# Creating a raster for results of Shapiro Test

In [ ]:
shapiro_values_2=area_show(shapiro_values)
print(shapiro_values_2)

In [ ]:
shapiro_image=[-9999]*12649
j=0
for i in mask_in:
    shapiro_image[i]=shapiro_values_2[j]
    j=j+1

In [ ]:
shapiro_image_values=numpy.array(shapiro_image)
shapiro_imageMatrix=shapiro_image_values.reshape(rows,col)
raster_shapiro=driver.Create("shapiro_1.tif",col,rows,1,gdal.GDT_Float32)
raster_shapiro.SetGeoTransform(img1.GetGeoTransform())
raster_shapiro.SetProjection(img1.GetProjection())
raster_shapiro.GetRasterBand(1).WriteArray(shapiro_imageMatrix)
del raster_shapiro